In [1]:
import os
import sys
import numpy as np
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ

In [3]:
# ------------------------------------------
# Create simulated data acquisition sources:
# ------------------------------------------
acq_simulated_data = LDAQ.simulator.SimulatedAcquisition(acquisition_name='sim_data', multi_processing=False)

In [4]:
# define simulated data using numpy array (array will be looped over):
fs = 10000
t = np.arange(fs*2)
x = np.sin(2*np.pi*50*t/fs) + np.random.randn(len(t))*0.2
y = np.sin(2*np.pi*100*t/fs + np.pi/2)
simulated_data = np.array([x, y]).T

# define simulated data using a function:
def simulate_signal(t, f1, f2):
    """
    first argument must be time.
    If multi_processing is True, then the function must be pickable and 
    all libraries must be imported inside the function.
    """
    import numpy as np
    sig1 = np.sin(2*np.pi*f1*t) + np.random.rand(len(t))*0.3
    sig2 = np.cos(2*np.pi*f2*t) + np.random.rand(len(t))*0.3
    return np.array([sig1, sig2]).T

In [5]:
# set simulated data (first argument is the function or array):
acq_simulated_data.set_simulated_data(simulated_data, channel_names=["ch1", "ch2"], sample_rate=fs, args=(84, 120)) # use array
# acq_simulated_data.set_simulated_data(simulate_signal, channel_names=["ch1", "ch2"], sample_rate=fs, args=(84, 120)) # use function

In [29]:
acq_simulated_data.set_trigger(level=0, channel=0, duration=10.)
acq_simulated_data.set_continuous_mode()

In [30]:
acq_simulated_data.run_acquisition(run_in_background=True)

In [47]:
_, data = acq_simulated_data.get_data(N_points="new")
data.shape, acq_simulated_data.is_running

((9870, 2), True)

In [48]:
acq_simulated_data.stop()

In [24]:
# ------------------------------------------
# create simulated video acquisition source:
# ------------------------------------------
acq_simulated_video = LDAQ.simulator.SimulatedAcquisition(acquisition_name='sim_video', multi_processing=False)

In [25]:
# define simulated video using numpy array:
fs = 30
simulated_video = np.random.randn(1000, 100, 100)

# define simulated video using a function:
def simulate_video(t):
    """
    first argument must be time.
    If multi_processing is True, then the function must be pickable and 
    all libraries must be imported inside the function.
    """
    import numpy as np
    N = t.shape[0]	
    return np.random.randn(N, 100, 100)

In [26]:
acq_simulated_video.set_simulated_video(simulated_video, channel_name_video='cam', sample_rate=fs) # use array
#acq_simulated_video.set_simulated_video(simulate_video, channel_name_video='cam', sample_rate=fs) # use function

In [27]:
# ----------------------------
# Create visulaization object:
# ----------------------------

vis = LDAQ.Visualization(sequential_plot_updates=False)
vis.add_lines((0,0), source='sim_data', channels=["ch1", "ch2"], refresh_rate=50)
vis.add_lines((1,0), source='sim_data', channels=["ch1", "ch2"], refresh_rate=500, function="fft")

vis.add_image(source="sim_video", channel="cam", refresh_rate=30)

vis.config_subplot((0,0), t_span=0.04, ylim=(-1.1, 1.1))
vis.config_subplot((1,0), t_span=0.5, ylim=(0, 1.1), xlim=(0, 500))

In [28]:
ldaq = LDAQ.Core(acquisitions=[acq_simulated_data, acq_simulated_video], visualization=vis)
ldaq.run(3.)

In [29]:
ldaq.get_measurement_dict()

{'sim_data': {'time': array([0.0000e+00, 1.0000e-04, 2.0000e-04, ..., 2.9997e+00, 2.9998e+00,
         2.9999e+00]),
  'channel_names': ['ch1', 'ch2'],
  'data': array([[ 1.00979685, -0.42577929],
         [ 0.94806224, -0.48175367],
         [ 1.24825977, -0.53582679],
         ...,
         [ 0.80880171, -0.24868989],
         [ 0.7779716 , -0.30901699],
         [ 0.99091336, -0.36812455]]),
  'sample_rate': 10000},
 'sim_video': {'time': array([0.        , 0.03333333, 0.06666667, 0.1       , 0.13333333,
         0.16666667, 0.2       , 0.23333333, 0.26666667, 0.3       ,
         0.33333333, 0.36666667, 0.4       , 0.43333333, 0.46666667,
         0.5       , 0.53333333, 0.56666667, 0.6       , 0.63333333,
         0.66666667, 0.7       , 0.73333333, 0.76666667, 0.8       ,
         0.83333333, 0.86666667, 0.9       , 0.93333333, 0.96666667,
         1.        , 1.03333333, 1.06666667, 1.1       , 1.13333333,
         1.16666667, 1.2       , 1.23333333, 1.26666667, 1.3       ,
    